# งานที่ 1a: ดำเนินการสร้างข้อความ

ในโน้ตบุ๊กนี้ คุณจะได้เรียนรู้วิธีใช้โมเดลภาษาขนาดใหญ่ (LLM) เพื่อสร้างการตอบกลับอีเมลหาลูกค้าที่ให้ข้อเสนอแนะเชิงลบเกี่ยวกับคุณภาพของการบริการลูกค้าที่ได้รับจากวิศวกรฝ่ายช่วยเหลือ ในโน้ตบุ๊กนี้ คุณจะสร้างอีเมลพร้อมคำขอบคุณโดยอิงจากอีเมลก่อนหน้าของลูกค้า คุณจะใช้โมเดล Amazon Titan โดยใช้ Amazon Bedrock API กับไคลเอ็นต์ Boto3

พรอมต์คำสั่งที่ใช้ในงานนี้เรียกว่าพรอมต์คำสั่งแบบ zero-shot ในพรอมต์คำสั่งแบบ zero-shot คุณจะอธิบายงานหรือเอาต์พุตที่ต้องการไปยังแบบจำลองภาษาเป็นภาษาธรรมดาๆ จากนั้นโมเดลจะใช้ความรู้และความสามารถที่ผ่านการฝึกไว้ล่วงหน้าเพื่อสร้างการตอบกลับ หรือทำงานให้เสร็จตามพรอมต์คำสั่งที่ให้เพียงอย่างเดียว

#### สถานการณ์
คุณคือ Bob ผู้จัดการฝ่ายบริการลูกค้าของ AnyCompany และลูกค้าบางคนไม่พอใจกับการบริการลูกค้าและกำลังให้ข้อเสนอแนะเชิงลบเกี่ยวกับบริการที่ให้โดยวิศวกรฝ่ายสนับสนุนลูกค้า ตอนนี้ คุณอยากตอบกลับลูกค้าเหล่านั้นเพื่อขอโทษสำหรับบริการที่ไม่ดีและเพื่อเรียกความไว้วางใจกลับคืนมา คุณต้องการความช่วยเหลือจาก LLM เพื่อสร้างอีเมลจำนวนมากให้แก่คุณสำหรับอีเมลที่ให้ความเป็นมิตรอย่างเป็นธรรมชาติและปรับให้เหมาะกับความรู้สึกของลูกค้าจากการติดต่อทางอีเมลก่อนหน้านี้

## งานที่ 1a.1: การตั้งค่าสภาพแวดล้อม

ในงานนี้ ให้ตั้งค่าสภาพแวดล้อมของคุณ

In [ ]:
#Create a service client by name using the default session.
import json
import os
import sys

import boto3
import botocore

module_path = ".."
sys.path.append(os.path.abspath(module_path))

bedrock_client = boto3.client('bedrock-runtime',region_name=os.environ.get("AWS_DEFAULT_REGION", None))

## งานที่ 1a.2: สร้างข้อความ

ในงานนี้ ให้คุณเตรียมอินพุตสำหรับบริการ Amazon Bedrock เพื่อสร้างอีเมล

In [ ]:
# create the prompt
prompt_data = """
Command: Write an email from Bob, Customer Service Manager, AnyCompany to the customer "John Doe" 
who provided negative feedback on the service provided by our customer support 
engineer"""

In [ ]:
body = json.dumps({
    "inputText": prompt_data, 
    "textGenerationConfig":{
        "maxTokenCount":8192,
        "stopSequences":[],
        "temperature":0,
        "topP":0.9
        }
    }) 

ถัดไป ให้คุณใช้โมเดล Amazon Titan

<i aria-hidden="true" class="fas fa-sticky-note" style="color:#563377"></i> **หมายเหตุ:** Amazon Titan รองรับหน้าต่างบริบทของโทเค็น ~4k และยอมรับพารามิเตอร์ต่อไปนี้
- `inputText`: พรอมต์คำสั่งไปยัง LLM
- `textGenerationConfig`: นี่คือพารามิเตอร์ที่โมเดลจะคำนึงถึงขณะสร้างเอาต์พุต

Amazon Bedrock API จะให้ API “invoke_model” แก่คุณซึ่งยอมรับสิ่งต่อไปนี้
- `modelId`: นี่คือโมเดล ARN สำหรับรุ่นพื้นฐานต่างๆ ที่มีอยู่ภายใต้ Amazon Bedrock
- “accept”: ประเภทของคำขออินพุต
- `contentType`: ประเภทเนื้อหาของเอาต์พุต
- `body`: สตริง json ที่ประกอบด้วยพรอมต์คำสั่งและการกำหนดค่า

โปรดดู [เอกสาร](https://docs.aws.amazon.com/bedrock/latest/userguide/model-ids-arns.html) สำหรับ Id โมเดลสร้างข้อความที่มีอยู่

## งานที่ 1a.3: เรียกโมเดลภาษา Amazon Titan Large

ในงานนี้ ให้คุณสำรวจว่าโมเดลสามารถสร้างเอาต์พุตตามพรอมต์คำสั่งที่สร้างขึ้นก่อนหน้านี้ได้อย่างไร

### สร้างเอาต์พุตให้เรียบร้อย

อีเมลนี้สร้างขึ้นโดยใช้โมเดล Amazon Titan โดยทำความเข้าใจในคำขออินพุตและใช้ความเข้าใจโดยธรรมชาติเกี่ยวกับรูปแบบต่างๆ คำขอไปยัง API เป็นแบบซิงโครนัสและรอให้เอาต์พุตทั้งหมดถูกสร้างโดยโมเดล

In [ ]:
#invoke model
modelId = 'amazon.titan-text-express-v1' # change this to use a different version from the model provider
accept = 'application/json'
contentType = 'application/json'
outputText = "\n"
try:

    response = bedrock_client.invoke_model(body=body, modelId=modelId, accept=accept, contentType=contentType)
    response_body = json.loads(response.get('body').read())

    outputText = response_body.get('results')[0].get('outputText')

except botocore.exceptions.ClientError as error:
    
    if error.response['Error']['Code'] == 'AccessDeniedException':
           print(f"\x1b[41m{error.response['Error']['Message']}\
                \nTo troubeshoot this issue please refer to the following resources.\
                 \nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\
                 \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n")
        
    else:
        raise error


In [ ]:
# The relevant portion of the response begins after the first newline character
# Below we print the response beginning after the first occurence of '\n'.

email = outputText[outputText.index('\n')+1:]
print(email)


### สตรีมการสร้างเอาต์พุต

Bedrock ยังสนับสนุนว่าเอาต์พุตสามารถสตรีมได้เนื่องจากถูกสร้างโดยโมเดลในรูปแบบของชิ้นส่วน อีเมลนี้ถูกสร้างขึ้นโดยการเรียกใช้โมเดลที่มีตัวเลือกสตรีมมิ่ง โดย `invoke_model_with_response_stream` จะส่งกลับ `ResponseStream` ซึ่งคุณสามารถอ่านได้

In [ ]:
# invoke model with response stream
output = []
try:
    
    response = bedrock_client.invoke_model_with_response_stream(body=body, modelId=modelId, accept=accept, contentType=contentType)
    stream = response.get('body')
    
    i = 1
    if stream:
        for event in stream:
            chunk = event.get('chunk')
            if chunk:
                chunk_obj = json.loads(chunk.get('bytes').decode())
                text = chunk_obj['outputText']
                output.append(text)
                print(f'\t\t\x1b[31m**Chunk {i}**\x1b[0m\n{text}\n')
                i+=1
            
except botocore.exceptions.ClientError as error:
    
    if error.response['Error']['Code'] == 'AccessDeniedException':
           print(f"\x1b[41m{error.response['Error']['Message']}\
                \nTo troubeshoot this issue please refer to the following resources.\
                 \nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\
                 \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n")
        
    else:
        raise error

สตรีมที่มีการตอบกลับช่วยให้ได้รับเอาต์พุตของโมเดลอย่างรวดเร็วและช่วยให้บริการสร้างการตอบกลับให้สมบูรณ์ในขณะที่คุณอ่าน วิธีนี้ช่วยในกรณีการใช้งานที่คุณขอให้โมเดลสร้างข้อความที่ยาวขึ้น ในภายหลังคุณสามารถรวมสิ่งที่สร้างขึ้นทั้งหมดเพื่อสร้างเอาต์พุตที่สมบูรณ์และใช้สำหรับกรณีการใช้งานของคุณ 

In [ ]:
#combine output chunks
print('\t\t\x1b[31m**COMPLETE OUTPUT**\x1b[0m\n')
complete_output = ''.join(output)
print(complete_output)


ตอนนี้คุณได้ทดลองใช้ boto3 SDK ซึ่งให้การเปิดเผยพื้นฐานกับ Amazon Bedrock API เมื่อใช้ API นี้ คุณได้เห็นกรณีการใช้งานของการสร้างอีเมลเพื่อตอบกลับความคิดเห็นเชิงลบของลูกค้า

### ลองด้วยตัวเอง
- เปลี่ยนพรอมต์คำสั่งตามฐานการใช้งานเฉพาะของคุณและประเมินเอาต์พุตของโมเดลต่างๆ
- เล่นด้วยความยาวโทเค็นเพื่อทำความเข้าใจเวลาแฝงและการตอบกลับของบริการ
- ใช้หลักการวิศวกรรมการโต้ตอบที่แตกต่างกันเพื่อให้ได้เอาต์พุตที่ดีขึ้น

### เก็บงาน

คุณทำโน้ตบุ๊กนี้เสร็จแล้ว หากต้องการย้ายไปยังส่วนถัดไปของแล็บ ให้ทำดังนี้

- ปิดไฟล์โน้ตบุ๊กนี้และไปต่อยัง **Task1b.ipynb**